# Téma 2: Frekventistické versus bayesovské modelování

Obsahem tohoto tématu bude náhled na principy frekventistického a bayesovského modelování. Jako příklad si vezmeme lineární regresi.

## Popis problému

Uvažujme, že pro jednotlivé časové okamžiky $t=1,2,\ldots$ máme k dispozici

- $y_t$ - **hodnoty závislé pozorované veličiny**,
- $x_t = [x_{t1}, \ldots, x_{tp}]$ - **hodnoty nezávislé pozorované veličiny**, např. časové indexy,

a že závislost mezi $y_t$ a $x_t$ je **lineární**. Dále uvažujeme, že tato závislost není plně deterministická, ale že je přítomen nepozorovatelný náhodný **šum** $\varepsilon_t$. Na jeho rozdělení nyní nebudeme klást konkrétní požadavky, s výjimkou toho, že má být **centrovaný** v nule $\mathbb{E}[\varepsilon_t]=0$, **nezávislý** a **stejně rozdělený** (iid) s rozptylem $\operatorname{var} \sigma^2$.

> Ještě než se do nějakého modelování pustíme, je vždy rozumné si vykreslit "scatterploty" a situaci posoudit "okometricky", neboť pouhé použití popisných statistik jako průměr, variance či korelační koeficient nemusí stačit. Oblíbeným příkladem tohoto úkazu je tzv. [Anscombeho kvartet](https://en.wikipedia.org/wiki/Anscombe%27s_quartet) - **čtyři datasety s prakticky či úplně stejnými hodnotami průměrů a variancí $x$ i $y$, se stejným korelačním koeficientem a rovnicí regresní přímky**, viz obrázek (zdroj:wikipedia).
![l2-anscombe.png](img/l2-anscombe.png)

## Stoupající objekt - sledování výšky v závislosti na čase

Měříme radarem výšku $y_t$ stoupajícího objektu s počáteční rychlostí v okamžiku záchytu $v_0$ a konstantním vertikálním zrychlením $a$. Výšku záchytu označíme jako nulovou v čase 1, tedy $y_1 = 0$. Chceme výšku modelovat jednoduše jako časovou řadu závislosti $y_t$ na čase. Jako první analytický krok si vykreslíme scatterplot.

![tracking](img/tracking-scatterplot2.png)

## Lineární model

Pokud budeme závislost $y_t$ na $x_t$ modelovat lineárním regresním modelem, budeme o proměnných mluvit rovněž jako o **regresandu** $y_t$ a **regresoru** $x_t$.

Nejběžnější lineární regresní model má tvar

$$
y_t = \beta_0 + \beta_1 x_{t1} + \ldots + \beta_p x_{tp} + \varepsilon_t = x_t^\intercal \beta + \varepsilon_t,
$$

kde $\beta_0, \ldots, \beta_p$ jsou **neznámé regresní koeficienty** a $\beta$ nazýváme **vektorem regresních koeficientů**. Pokud uspořádáme $T$ takových proměnných do příslušných vektorů a matic, dostáváme

$$
y = X\beta + \varepsilon,
$$

kde

$$
y =
\begin{bmatrix}
y_1 \\ y_2 \\ \vdots \\ y_T
\end{bmatrix},
\qquad
\varepsilon =
\begin{bmatrix}
\varepsilon_1 \\ \varepsilon_2 \\ \vdots \\ \varepsilon_T
\end{bmatrix},
\qquad
X =
\begin{bmatrix}
x_1^\intercal \\ x_2^\intercal \\ \vdots \\ x_T^\intercal
\end{bmatrix}
=
\begin{bmatrix}
1, x_{11}, x_{12}, \ldots, x_{1p} \\ 
1, x_{21}, x_{22}, \ldots, x_{2p} \\ 
\vdots \\ 
1, x_{T1}, x_{T2}, \ldots, x_{Tp} \\ 
\end{bmatrix},
\qquad
\beta =
\begin{bmatrix}
\beta_0 \\ \beta_1 \\ \vdots \\ \beta_p
\end{bmatrix}
$$

Uvedený lineární regresní model samozřejmě není jediný možný. Můžeme uvažovat například následující:

- $y_t = \beta_{1} x_{t1} + \ldots + \beta_{p} x_{tp} + \varepsilon_t$
- $y_t = \beta_0 + \beta_{1} x_{t1} + \ldots + \beta_{p} x_{tp} + \varepsilon_t$
- $y_t = \beta_0 + \beta_{1} x_{t} + \beta_{2} x_{t}^2 + \ldots + \beta_{p} x_{t}^{p} + \varepsilon_t$

atd.


**K zamyšlení:**

- identifikujte jednotlivé regresní modely
- patří uvedené modely také mezi lineární?
- zapište matice $X$ uvedených modelů. Platí linearita?
- mohli bychom do modelů výše vpravit např. logaritmy? Zachováme linearitu modelu?


### Odhad $\beta$
Máme-li vybraný (potenciálně) vhodný model, je prvotním úkolem **odhad regresních koeficientů $\beta$**. Díky tomu budeme mít kompletní vztah mezi regresorem $x_t$ a pozorováními $y_t$. To umožní například provést odhad (predikci) budoucího $y_{t+1}$ na základě znalosti $x_{t+1}$, nebo nějaké hodnoty pozorované veličiny v minulosti.

Pozorným pohledem na matici $X$ vidíme, že pro $p+1$ neznámých koeficientů $\beta_i$ máme $T$ pozorování, kde zpravidla $T\geq p+1$. To znamená, že máme **víc rovnic než neznámých**, systém je předeterminovaný a **nemá žádné řešení**! 

# Frekventistický přístup k lineární regresi

Budeme odhadovat regresní koeficienty $\beta$ metodou nejmenších čtverců. Vyjádříme-li součet čtverců

$$
S(\beta) = \sum_{t=1}^{T} \left( y_t - \sum_{j=0}^{p} x_{tj}\beta_{j}\right)^2 
= \big|\big|y - X\beta \big|\big|^2,
$$

potom naším cílem je tento součet **minimalizovat**,

$$
\widehat{\beta} = \arg\min_{\beta} S(\beta).
$$

Tato minimalizace je v pozadí **metody nejmenších čtverců**.

Označme odhad $\beta=[\beta_0, \ldots, \beta_p]^\intercal$ jako $\widehat\beta=[\hat\beta_0, \ldots, \hat\beta_p]^\intercal$. Potom platí následující věta:

> **Věta**
>
> Pro odhad platí $\widehat\beta = (X^\intercal X)^{-1} X^\intercal y$.
>
> **Důkaz**
>
> Napišme $S(\beta)$ ve tvaru
>
>$$
S(\beta) = \left[(y-X\widehat\beta) + (X\widehat\beta - X\beta) \right]^\intercal \left[(y-X\widehat\beta) + (X\widehat\beta - X\beta) \right].
$$
>
> Roznásobením a využitím $X^\intercal (y - X\widehat\beta) = 0$ z tvrzení věty dostaneme
> 
>$$
S(\beta) = (y - X\widehat\beta)^\intercal (y - X\widehat\beta) + (\widehat\beta-\beta)^\intercal X^\intercal X (\widehat\beta-\beta).
$$
>
> Jelikož matice $X^\intercal X$ je pozitivně definitní (proč?), platí
>
>$$
(\widehat\beta-\beta)^\intercal X^\intercal X (\widehat\beta-\beta) \geq 0.
$$
> Rovnost nastává při $\widehat\beta = \beta$.

Odhad lze získat ze **soustavy normálních rovnic** kterou dostaneme z rovnice $y = X\widehat\beta$ regularizací - násobením $X^\intercal$ zprava,

$$
X \widehat\beta = y \qquad \Longrightarrow \qquad X^\intercal X \widehat\beta = X^\intercal y.
$$

> **Definice**
>
> **Reziduální součet čtverců, regresní součet čtverců, celkový součet čtverců** a **reziduální rozptyl** jsou hodnoty
>
>$$
\begin{aligned}
SSE &= \big|\big| y - X\widehat\beta \big|\big|^2, \\
SSR &= \big|\big| \hat{y} - \bar{y} \big|\big|^2, \\
SST &= \big|\big| y - \bar{y}\big|\big|^2 = SSR + SSE,\\
s^2 &= \frac{SSE}{T-p+1}.
\end{aligned}
$$
>
> kde $\bar{y}$ je střední hodnota (resp. průměr) pozorování a $\widehat{y}$ je regresní odhad (predikce) hodnot pozorování z modelu.

Povšimněme si, že platí následující (nebudeme zavádět jako extra větu):

$$
\begin{aligned}
\mathbb{E}[\widehat\beta] &= (X^\intercal X)^{-1} X^\intercal \mathbb{E}[y] = (X^\intercal X)^{-1} X^\intercal X \beta = \beta,\\
\operatorname{var} \widehat\beta &= (X^\intercal X)^{-1} X^\intercal \sigma^2 I X (X^\intercal X)^{-1},\\
\mathbb{E}{s^2} &= \sigma^2.
\end{aligned}
$$

První dvě tvrzení jsou rovnou odvozena, odvození posledního pro jednoduchost přeskočíme. Řekneme jen, že takovýto odhad rozptylu $\sigma^2$ je **nestranný**.

Jako diagnostický aparát pro zjištění vhodnosti regresního modelu lze použít testování hypotéz o odhadech regresních koeficientů, zhodnocení korelačního koeficientu, scatterplot, nebo třeba [koeficient determinace](https://en.wikipedia.org/wiki/Coefficient_of_determination)

$$
R^2 = \frac{SSR}{SST} = 1 - \frac{SSE}{SST} \in [0, 1].
$$

Čím je $R^2$ blíže jedné, tím lépe model vystihuje závislost mezi regresorem a regresandem. Má ale své nevýhody, neboť neříká, zda:

- regresor je příčinou změn regresandu,
- je regresní model vůbec vhodný,
- neexistuje multikolinearita v regresorech,
- máme dostatečný rozsah dat pro regresi a zda byla využita vhodná data.

Abychom potlačili jev, kdy přidávání nových regresorů automaticky zvyšuje $R^2$, používáme často adjustovaný koeficient determinace

$$
R_{adj}^2 = 1 - (1-R^2) \frac{n-1}{n-p-1}.
$$

**K zamyšlení**
- odhad $\beta$ plyne rovněž z minimalizace součtu čtverců $S(\beta)$ pomocí derivace. Zkuste.
- co označuje reziduální součet čtverců?

_Za povšimnutí rozhodně stojí, že $S(\beta)$ bychom při předpokladu normality $\varepsilon_t \sim \mathcal{N}(0, \sigma^2)$ našli v exponentu hustoty normálního rozdělení_

$$
f(y_t|\beta, X, \sigma^2) = \frac{1}{(2\pi\sigma^2)^{T/2}} \exp\left\{ -\frac{1}{2\sigma^2} \sum_{t=1}^T \left( y_t - \sum_{j=0}^{p} x_{tj}\beta_{j}\right)^2  \right\}.
$$

_Tento předpoklad nutně nemáme, aspoň ne ve frekventismu. Ale je-li splněn, bude mít odhad **metodou nejmenších čtverců** dobré vlastnosti z hlediska metody maximální věrohodnosti._

### Konkrétní příklad - jednoduchá lineární regrese

Podívejme se na jednoduchou lineární regresi modelem

$$
y_t = \beta_0 + \beta_1 x_t + \varepsilon_t, \qquad \text{iid}\quad \varepsilon_t \sim \mathcal{N}(0, \sigma^2), \qquad t=1, \ldots, T.
$$

V časových řadách by takovým $x_t$ mohla být např. minulá naměřená hodnota, $y_{t-1}$. Vypočtením odhadů $\beta=[\beta_0, \beta_1]^\intercal$ v podobě $\widehat\beta=[\hat\beta_0, \hat\beta_1]^\intercal$ dostaneme regresní přímku

$$
\widehat{y_t} = \hat\beta_0 + \hat\beta_1 x_t, \qquad t=1,\ldots, T,
$$

Jednotlivá rezidua (chyby) regrese můžeme určit jako

$$
e_t = y_t - \widehat{y}_t, \qquad t=1,\ldots,T.
$$

Nyní bychom měli model analyzovat, např. testovat

$$
\begin{aligned}
H_0&: \beta_1 = 0, \\
H_1&: \beta_1 \ne 0.
\end{aligned}
$$

Pak vypočteme testovací statistiku

$$
T = \frac{\hat\beta_1}{s} \sqrt{\sum_{t=1}^T x_t^2 - n\bar{x}^2}
$$

a $H_0$ zamítáme, pokud je překročena příslušná kritická hodnota Studentova rozdělení, $|T| \geq t_{n-2}(\alpha)$.

Je možné rovněž zkonstruovat $(1-\alpha)$ interval spolehlivosti pro $\beta_0 + \beta_1 x$, který s pravděpodobností $1-\alpha$ překrývá hodnotu $\beta_0 + \beta_1 x$. Ten tvoří dvě větve hyperboly, které dohromady tvoří **pás spolehlivosti kolem regresní přímky** a je platný pro **každý jednotlivý bod přímky**, nikoliv však pro celou přímku. V tomto případě má tvar

$$
\hat\beta_0 + \hat\beta_1 x_t \pm t_{n-2}(\alpha) s \sqrt{\frac{1}{n} + \frac{(x_t - \bar{x})^2}{\sum_{t=1}^T x_t^2 - n\bar{x}^2}}.
$$

Predikce pro zadané $x$ vychází celkem předvídatelně z modelu

$$
\hat{y} = \hat\beta_0 + \hat\beta_1 x
$$

a stejně jako v předchozím případě je doprovázena intervalem spolehlivosti, zde

$$
\hat{y} \pm t_{n-2}(\alpha) s \sqrt{1 + \frac{1}{n} + \frac{(x - \bar{x})^2}{\sum_{t=1}^T x^2 - n\bar{x}^2}}
$$

## Ad příklad s výškou, fitování přímky

$$y_t = \beta_0 + \beta_1 x_t + \varepsilon_t = \beta_0 + \beta_1 t + \varepsilon_t$$
![line](img/tracking-plot-line.png)
```
                            OLS Regression Results                          
==============================================================================
Dep. Variable:                 height   R-squared:                       0.956
Model:                            OLS   Adj. R-squared:                  0.955
Method:                 Least Squares   F-statistic:                     1694.
Date:                Mon, 08 Oct 2018   Prob (F-statistic):           1.16e-54
Time:                        07:10:33   Log-Likelihood:                -497.02
No. Observations:                  80   AIC:                             998.0
Df Residuals:                      78   BIC:                             1003.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -257.6964     27.093     -9.512      0.000    -311.634    -203.759
time          24.3715      0.592     41.159      0.000      23.193      25.550
==============================================================================
Omnibus:                        8.088   Durbin-Watson:                   0.102
Prob(Omnibus):                  0.018   Jarque-Bera (JB):                6.067
Skew:                           0.553   Prob(JB):                       0.0481
Kurtosis:                       2.227   Cond. No.                         90.7
==============================================================================
```

V tabulce ze `statsmodels` se dozvíme o 
- počtu dat ($n$)
- stupních volnosti (Df) reziduí ($n-p-1$, máme-li offset, jinak $n-p$) a modelu ($p$)
- typ odhadu kovariance (robustní či nerobustní)
- [koeficient determinace](https://en.wikipedia.org/wiki/Coefficient_of_determination) $R^2$ a adjustovaný koeficient determinace
- F statistiku a p-hodnotu F testu (H0: každý koeficient je odlišný od 0)
- log-likelihood dat při daném odhadu, tj. logaritmus (hustoty) pravděpodobnosti dat při odhadu $\widehat\beta$ a vybraném modelu, $p(y_1,\ldots, y_n|\widehat\beta, \text{model})$
- [Akaikeho informační kritérium](https://en.wikipedia.org/wiki/Akaike_information_criterion) - pro $p$ parametrů a likelihood dat $L$ je $AIC = 2p - 2\ln (L)$
- [Bayesovské informační kritérium](https://en.wikipedia.org/wiki/Bayesian_information_criterion) $BIC = p\ln(n) - 2\ln(L)$. V obou případech čím nižší hodnota kritéria, tím lepší model. Rozdíl je v penalizaci (a filosofii, viz odkazy),
- odhady koeficientů vč. statistik intervalů spolehlivosti a testů (H0: koef. je nulový)
- [Omnibus test](http://en.wikipedia.org/wiki/Omnibus_test) normality reziduí (H0: normálně rozdělená rezidua)
- [Šikmost](https://en.wikipedia.org/wiki/Skewness) a [špičatost](https://en.wikipedia.org/wiki/Kurtosis) rozdělení reziduí
- [Durbin-Watsonova statistika](https://www.statsmodels.org/stable/generated/statsmodels.stats.stattools.durbin_watson.html#statsmodels.stats.stattools.durbin_watson) vypovídající o autokorelaci reziduí (2 = žádná autokorelace, menší a větší hodnota pro kladnou a zápornou autokorelaci)
- [Jarque-Bera test normality reziduí](https://en.wikipedia.org/wiki/Jarque%E2%80%93Bera_test) - čím vyšší hodnota, tím dál od normality, p-hodnota potom indikuje výsledek testu
- [Podmíněnost regresní matice](https://en.wikipedia.org/wiki/Multicollinearity) indikující multikolinearitu, tj. lineární závislost regresorů.

> **Malé zavzpomínání na testování hypotéz**
>
> Mějme nulovou hypotézu $H_0$, hladinu významnosti $\alpha$ a vypočtenou $p$-hodnotu. Potom
>
> $$p \leq \alpha \Rightarrow\text{zamítáme $H_0$ na hladině $\alpha$}.$$
>
> Hladinu významnosti obvykle volíme 0.05, alternativně i 0.01 či jinak.

## Ad příklad s výškou, fitování paraboly

$$y_t = \beta_0 + \beta_1 x_t^2 + \varepsilon_t = \beta_0 + \beta_1 t^2 + \varepsilon_t$$
![parabole](img/tracking-plot-parab.png)

```
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 height   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.995
Method:                 Least Squares   F-statistic:                 1.595e+04
Date:                Mon, 08 Oct 2018   Prob (F-statistic):           5.65e-92
Time:                        08:05:06   Log-Likelihood:                -408.92
No. Observations:                  80   AIC:                             821.8
Df Residuals:                      78   BIC:                             826.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     67.4603      6.793      9.931      0.000      53.937      80.983
time           0.3045      0.002    126.310      0.000       0.300       0.309
==============================================================================
Omnibus:                        0.689   Durbin-Watson:                   0.866
Prob(Omnibus):                  0.709   Jarque-Bera (JB):                0.519
Skew:                           0.197   Prob(JB):                        0.771
Kurtosis:                       2.979   Cond. No.                     4.21e+03
==============================================================================
```

# Bayesovský přístup k lineární regresi

## Trocha bayesovské teorie úvodem

Bayesovství se při modelování opírá o **apriorní distribuci, jež kvantifikuje naši znalost o neznámých parametrech před zohledněním pozorovaných dat**. Parametry obvykle označujeme pod souhrnným $\theta$. V naší regresi tedy půjde o $\theta = (\beta, \sigma^2)$. Na parametry tedy nahlížíme jako na **náhodné veličiny**, ale pozor, náhoda - neurčitost - se spojena s naší znalostí o parametru. Ten sám je pevné číslo, stejně jako ve frekventismu.

> **Věta (Bayesova)**
>
> Nechť $f(y_t|x_t, \theta)$ je hustota pravděpodobnosti $y_t|x_t,\theta$. Dále nechť $\pi(\theta|y_{0:t-1}, x_{0:t-1})$ je apriorní hustota pro $\theta$. Aposteriorní hustota pro jednokrokový update má tvar
>
$$
\begin{aligned}
\pi(\theta|y_{0:t}, x_{0,t}) 
&= 
\frac
{f(y_t|x_t, \theta) \pi(\theta|x_{0:t-1}, y_{0:t-1})}
{\int f(y_t|x_t, \theta) \pi(\theta|x_{0:t-1}, y_{0:t-1})d\theta} \\
&=
\frac
{f(y_t|x_t, \theta) \pi(\theta|x_{0:t-1}, y_{0:t-1})}
{f(y_t|x_t)} \\
&\propto
f(y_t|x_t, \theta) \pi(\theta|x_{0:t-1}, y_{0:t-1}).
\end{aligned}
$$

Poslední řádek bude náš nejoblíbenější, neboť vyjadřuje bayesovský update v jednoduchém zápisu - bez normalizující hustoty ve jmenovateli. Ta je totiž nezávislá na odhadovaném parametru $\theta$. Normalizaci si (*teoreticky*) snadno zařídíme kdykoliv.

Bayesovským odhadem $\theta$ je tedy **aposteriorní distribuce**. Jako **bodové odhady** můžeme použít například míry polohy v této distribuci,

- střední hodnotu $\mathbb{E}[\theta]$, přesněji $\mathbb{E}[\theta|x_{0:t}, y_{0:t}]$, nebo 
- modus (maximum hustoty psti) - tzv. maximum aposteriori (MAP) odhad, nebo 
- medián (prostřední hodnotu).

Jak moc tomuto odhadu věříme, neboli jak mnoho informace aposteriorní distribuce obsahuje, nám udává například rozptyl, $\text{var}\theta$.

Odvoďme si nyní bayesovský update pro více dat současně:
$$
\begin{aligned}
\pi(\theta|y_{0:1}, x_{0:1})
&\propto
f(y_1|x_1, \theta) \pi(\theta|x_{0}, y_{0}) \\
\pi(\theta|y_{0:2}, x_{0:2})
&\propto
f(y_2|x_2, \theta) \pi(\theta|x_{0:1}, y_{0:1}) \\
&\propto \pi(\theta|x_{0}, y_{0}) f(y_1|x_1, \theta) f(y_2|x_2, \theta) \\
&\vdots \\
\pi(\theta|y_{0:t}, x_{0:t})
&\propto
\pi(\theta|y_{0}, x_{0})
\prod_{\tau = 1}^{t}
f(y_{\tau}|x_{\tau}, \theta) \\
&\propto
\pi(\theta|y_{0:\tau-1}, x_{0:\tau-1})
\prod_{\tau = \widetilde{\tau}}^{t}
f(y_{\widetilde\tau}|x_{\widetilde\tau}, \theta),
\end{aligned}
$$
kde předposlední řádek říká, že můžeme pronásobit modely a pak teprve updatovat počáteční apriorno a poslední řádek říká, že totéž můžeme udělat v kterémkoliv čase. 

Závěr: **sekvenční update po jednom datu je stejný, jako update více daty najednou**. Opět radikálně jiný přístup, než ve frekventistickém pojetí. Bayesovství nám navíc implicitně nabízí (teoreticky jednoduchou) cestu, jak odhady sekvenčně vylepšovat, když získáme nová data.

### Sekvenční odhad

Výše jsme si ukázali, že teoreticky je sekvenční bayesovský update jednoduchá záležitost. Použijeme apriorní distribuci, přidáme nové pozorování, dostaneme aposteriorní a tu opět použijeme jako apriorní pro další pozorování:

$$
\pi(\theta|x_{0}, y_{0}) \xrightarrow[\text{Bayes}]{x_1, y_1}
\pi(\theta|x_{0:1}, y_{0:1}) \xrightarrow[\text{Bayes}]{x_2, y_2}
\pi(\theta|x_{0:2}, y_{0:2}) \rightarrow
\cdots \xrightarrow[\text{Bayes}]{x_t, y_t}
\pi(\theta|x_{0:t}, y_{0:t}) \rightarrow
\cdots
$$
![bayes-evol](img/bayes-evol.png)

**Ač se to ze vzorců výše možná nezdá, zásadním problémem bayesovského odhadu (modelování) je odvození aposteriorní distribuce a jejích vlastností. A se sekvenčním odhadem se tento problém ještě umocňuje. Pokud po prvním updatu není aposteriorno některou z "běžných" distribucí, pak s dalším updatem je aposteriorno ještě komplikovanější.**

**Existuje ale případ, kdy výpočty půjdou velmi pohodlně. K tomu si musíme zavést exponenciální třídu distribucí a konjugovaná apriorna.**

> **Definice (exponenciální třída distribucí)**
>
> Uvažujme náhodnou veličinu $y$ podmíněnou veličinou $x$ a parametrem $\theta$. Exponenciální třída distribucí obsahuje distribuce s hustotou pravděpodobnosti ve tvaru
> $$
f(y|x, \theta) = h(y, x) g(\theta) \exp \left[ \eta^{\intercal} T(y,x) \right],
$$
>
> kde $\eta \equiv \eta(\theta)$ je přirozený parametr, $T(y,x)$ je suficientní statistika fixního rozměru, $h(y,x)$ je známá funkce a $g(\theta)$ je normalizační funkce. Platí-li $\eta(\theta)=\theta$ je třída kanonická.


> **Definice (konjugovaná apriorní distribuce)**
>
> Nechť $y|x, \theta$ má rozdělení z exponenciální třídy distribucí. Říkáme, že apriorní distribuce $\theta$ s hyperparametry $\xi$ a $\nu$ je k němu konjugovaná, pokud její hustota pravděpodobnosti má tvar
$$
        \pi(\theta) = q(\xi, \nu) g(\theta)^{\nu} \exp \left[ \eta^{\intercal} \xi \right],
$$
>
>kde $\xi$ má stejný rozměr jako $T(y,x)$, $\nu\in\mathbb{R}^{+}$ a $q(\xi,\nu)$ je známá funkce. Funkce $g(\theta)$ je stejná jako normalizační funkce v hustotě pro $y|x, \theta$.

Hyperparametry nazýváme parametry apriorna, abychom je slovně odlišili od parametrů modelu. Má-li apriorno rovněž své apriorno, někteří označují jeho parametry jako hyper-hyperparametry (ale to už je trochu přitažené).

### Bayesovský odhad s konjugovaným apriornem
Použijeme-li konjugovaná apriorna s hyperparametry $\xi_{t-1}$ a $\nu_{t-1}$, je Bayesův update

$$
\pi(\theta|y_{0:t}, x_{0,t}) 
\propto
f(y_t|x_t, \theta) \pi(\theta|x_{0:t-1}, y_{0:t-1})
$$

vlastně jen triviální součet

$$
\begin{aligned}
    \xi_{t} &= \xi_{t-1} + T(y_{t},x_{t}), \\
    \nu_{t} &= \nu_{t-1} + 1.
\end{aligned}
$$

---

Při více datech současně má bayesovský update tvar
$$
\pi(\theta|y_{0:1}, x_{0:1})
\propto
\pi(\theta|y_{0:\tau-1}, x_{0:\tau-1})
\prod_{\tau = \widetilde{\tau}}^{t}
f(y_{\widetilde\tau}|x_{\widetilde\tau}, \theta),
$$
což není nic jiného, než opět součet
$$
\begin{aligned}
    \xi_{t} &= \xi_{\tau-1} + \sum_{\widetilde{\tau}=\tau}^{t} T(y_{\widetilde{\tau}},x_{\widetilde{\tau}}),\\
    \nu_{t} &= \nu_{\tau-1} + t - \tau+1.
\end{aligned}
$$

**Závěr: Bayesova věta se při použití konjugovaného apriorna ztriviální na přičtení suficientní statistiky k hyperparametru $\xi_{t-1}$ a inkrementaci hyperparametru $\nu_{t-1}$.**

## Bayesovská lineární regrese (konečně)

### Model
Uvažujme znovu lineární model pro skalární reálné $y_t$ determinované regresním vektorem $x_t\in\mathbb{R}^{p+1}$ a vektorem regresních koeficientů $\beta\in\mathbb{R}^{p+1}$ ve tvaru

$$
\begin{aligned}
y_t &= \beta^\intercal x_t + \varepsilon_t, \\
\varepsilon_t &\sim \mathcal{N}(0, \sigma^2) \qquad \text{iid}.
\end{aligned}
$$

Oproti frekventistické kapitolce jsme prohodili pořadí $x_t$ a $\beta$, ale jelikož je $y_t$ skalární, nic se tím nemění. Mohli bychom uvažovat i původní pořadí, ale zde se nám bude víc hodit toto.

Jelikož měření jsou zatížena normálním šumem, model je tedy ekvivalentně normální, $y_t\sim\mathcal{N}(\beta^\intercal x_t, \sigma^2)$ a má hustotu

$$
\begin{aligned}
    f(y_{t}|x_{t}, \beta, \sigma^{2}) 
    &= \frac{(\sigma^{2})^{-\frac{1}{2}}}{\sqrt{2\pi}}
       \exp
       \left\{ 
           -\frac{1}{2\sigma^{2}} (y_{t} - \beta^{\intercal}x_{t})^{2} 
       \right\} \notag \\
    &= \frac{(\sigma^{2})^{-\frac{1}{2}}}{\sqrt{2\pi}}
       \exp
       \Bigg\{ 
           \text{Tr}
           \bigg( 
               \underbrace{
                   -\frac{1}{2\sigma^{2}}
                   \begin{bmatrix}
                       1 \\ -\beta
                   \end{bmatrix}
                   \begin{bmatrix}
                       1 \\ -\beta
                   \end{bmatrix}^{\intercal}
               }_{\eta}
               \underbrace{
                   \begin{bmatrix}
                       y_{t} \\ x_{t}
                   \end{bmatrix}
                   \begin{bmatrix}
                       y_{t} \\ x_{t}
                   \end{bmatrix}^{\intercal}
               }_{T(y_{t}, x_{t})}    
           \bigg)
       \Bigg\}.
\end{aligned}
$$

Nyní zbývá pro tento model najít vhodné apriorno, ideálně konjugované.

### Apriorní distribuce
Jelikož neznáme ani regresní koeficienty ve vektoru $\beta$, ani varianci šumu $\sigma^2$, budeme je odhadovat. Vhodná konjugovaná apriorní distribuce $\pi(\beta, \sigma^2)$ je **normální inverzní-gama** distribuce,

$$
\beta, \sigma^{2} 
\sim \mathcal{N}i\mathcal{G}(m_{t-1}, V_{t-1}, a_{t-1}, b_{t-1})
= \underbrace{\mathcal{N}(m_{t-1}, \sigma^{2} V_{t-1})}_{\pi(\beta|\sigma^2)} 
\times 
\underbrace{i\mathcal{G}(a_{t-1}, b_{t-1})}_{\pi(\sigma^2)},
$$

s reálnými hyperparametry $a_{t-1}>0$ a $b_{t-1}>0$, vektorem středních hodnot $m_{t-1}\in\mathbb{R}^{p+1}$ a škálovací maticí $V_{t-1}^{-1}$ odpovídající dimenze. Jak mohou například hustoty - normální a inverzní gama - takového apriorna vypadat ukazuje obrázek. Výsledná NiG hustota je samozřejmě komplikovanější.
![N x iG](img/l2-apriorno-nig.jpg)

Jen pro zajímavost si ukažme hustotu pravděpodobnosti takové distribuce:

$$
\pi(\beta, \sigma^{2}|\cdot)
    = \frac{b^{a_{t-1}} (\sigma^{2})^{-(a_{t-1}+1+\frac{p}{2})}}{\sqrt{2\pi}|V_{t-1}|^{\frac{1}{2}}\Gamma(a_{t-1})}
       \exp
       \Bigg\{ 
           -\frac{1}{2\sigma^{2}}
           \bigg[ 
           b_{t-1} + 
               \text{Tr}
               \bigg( 
                       \begin{bmatrix}
                           1 \\ -\beta
                       \end{bmatrix}^{\intercal}
                       \begin{bmatrix}
                           1 \\ -\beta
                       \end{bmatrix}
                       \begin{bmatrix}
                           m_{t-1}^{\intercal} \\ I 
                       \end{bmatrix}
                       V_{t-1}^{-1}
                       \begin{bmatrix}
                           m_{t-1}^{\intercal} \\ I 
                       \end{bmatrix}^{\intercal}
                \bigg)
            \bigg]
       \Bigg\}.
$$

Porovnáním s hustotou modelu lze dovodit, že pro nás je důležité určit hyperparametry $\xi_{t-1}$ a $\nu_{t-1}$, jež zastoupí naše $a_{t-1}, b_{t-1}, m_{t-1}$ a $V_{t-1}$. Ty jsou

$$
\begin{aligned}
    \xi_{t-1} 
    &=
    \begin{bmatrix}
        m_{t-1}^{\intercal} V_{t-1}^{-1} m_{t-1} + 2b_{t-1} & m_{t-1}^{\intercal} V_{t-1}^{-1} \\
        V_{t-1}^{-1}m_{t-1} & V_{t-1}^{-1}
    \end{bmatrix} \\
    &=
    \begin{bmatrix}
        \xi_{t-1}^{[11]} & \xi_{t-1}^{[12]} \\
        \xi_{t-1}^{[21]} & \xi_{t-1}^{[22]}
    \end{bmatrix}, \\
\nu_{t-1} &= 2a_{t-1}.
\end{aligned}
$$

A ty už jsou pro nás zajímavé, neboť zredukují komplikovaný bayesovský update na jednoduchý součet.

### Bayesovský update
Bayesovský update - součet hyperparametru $\xi_{t-1}$ se suficientní statistikou $T(y_t, x_t)$ - nám pak odhalí aposteriorní distribuci a relativně snadnou algebrou pak dostaneme

$$
\begin{aligned}
    V_{t} &= \left( V_{t-1}^{-1} + x_{t}x_{t}^{\intercal} \right)^{-1}
           = V_{t-1} - \frac{V_{t-1} x_{t}x_{t}^{\intercal} V_{t-1}}{1+x_{t}^{\intercal} V_{t-1} x_{t}}= \left(\xi_{t}^{[22]}\right)^{-1}, \\ 
    m_{t} &= V_{t}(V_{t-1}^{-1}m_{t-1} + y_{t}x_{t}) = \left(\xi_{t}^{[22]}\right)^{-1} \xi_{t}^{[21]}, \\
    a_{t} &= a_{t-1} + \frac{1}{2} = \frac{1}{2}(\nu_{t-1} + 1) = \frac{1}{2}\nu_{t}, \label{eq:nig-update} \\
    b_{t} &= b_{t-1} + \frac{1}{2} \left(-m_{t}^{\intercal}V_{t}^{-1}m_{t} + m_{t-1}^{\intercal} V_{t-1}^{-1}m_{t-1} + y_{t}^{2} \right) \\
    &= \frac{1}{2}\left[\xi_{t}^{[11]} - \xi_{t}^{[12]}\left( \xi_{t}^{[22]} \right)^{-1} \left( \xi_{t}^{[12]} \right)^{\intercal}\right], \notag
\end{aligned}
$$

Není bez zajímavosti, že tento přístup k nalezení aposteriorních hyperparametrů je nemálo snadnější, než jejich odvozování z původní podoby, které je ale v literatuře běžné.

Připomeňme, co zaznělo výše:
$$
\beta, \sigma^{2} = 
\underbrace{\mathcal{N}(m_{t-1}, \sigma^{2} V_{t-1})}_{\pi(\beta|\sigma^2)} 
\times
\underbrace{i\mathcal{G}(a_{t-1}, b_{t-1})}_{\pi(\sigma^2)}.
$$

**Odhady dostaneme z marginálních distribucí:**
- $\hat{\sigma}^2 = \frac{b_{t}}{a_{t}-1}$. To plyne z marginální [inverzní gama distribuce](https://en.wikipedia.org/wiki/Inverse-gamma_distribution). Variance je $\operatorname{var}(\sigma^{2}|\cdot) = \frac{b_{t}^{2}}{(a_{t}-1)^{2}(a_{t}-2)}$. **Neurčitost - měřená variancí - plyne z konečného počtu měření a s $t\to\infty$ jde k nule.**
- $\hat{\beta} = m_t$, což dostaneme z marginální distribuce $\int \pi(\beta|\sigma^2) \pi(\sigma^2)d\sigma^2$ jež je [Studentova t distribuce](https://en.wikipedia.org/wiki/Student%27s_t-distribution#Non-standardized_Student%27s_t-distribution) s $2a_t$ stupni volnosti, centrovaná ve zmíněném $m_t$ a se škálovací maticí $\frac{b_t}{a_t}V_t$. Variance odhadu $var \beta = \frac{b_t}{a_t-1}V_t$. **Neučitost plyne z konečného počtu měření a z přítomnosti šumu $\varepsilon_t$ s variancí $\sigma^2$.**

### Predikce
Předpokládejme, že se ptáme na hodnotu $y'$ pro zadané $x'$, například budoucí hodnotu. Jako bayesovci na to půjdeme přes *prediktivní hustotu*. Model vynásobíme aposteriorní hustotou a vyintegrujeme parametry:
$$
f(y'|y_{0:t},x_{0:t},x') = \iint f(y'|x', \beta, \sigma^{2}) \pi(\beta, \sigma|y_{0:t}, x_{0:t}) \mathrm{d}\beta \mathrm{d}\sigma^{2}.
$$

To je opět [Studentova t distribuce](https://en.wikipedia.org/wiki/Student%27s_t-distribution#Non-standardized_Student%27s_t-distribution)
$$
y'|y_{0:t}, x_{0:t}, x' \sim t_{2a_{t}}\left(m_{t}^{\intercal}x', \frac{b_{t}}{a_{t}} \left(1 + (x')^{\intercal}V_{t}x'\right) \right).
$$

Co vidíme? Predikce je centrována v $m_t^{\intercal} x' = \hat{\beta}^\intercal x'$, přesně jak bychom čekali. Oproti frekventismu máme navíc i informaci, jak moc jsme si tou polohou jisti - $var(y'|\cdot) = \frac{b_t}{a_t-1} \left(1 + (x')^{\intercal}V_{t}x'\right)$. **Neurčitost plyne z neurčitosti v odhadu $\beta$ a odhadu $\sigma^2$.**

## Ad příklad s výškou, fitování paraboly bayesovsky

Začneme z velmi plochého apriorna centrovaného pro regresní koeficienty v 0 a fitujeme

$$y_t = \beta_0 + \beta_1 x_t^2 + \varepsilon_t = \beta_0 + \beta_1 t^2 + \varepsilon_t$$
![parabole](img/tracking-bayes.png)

```
Final estimate of beta:  [57.66874799  0.29756   ]
Final estimate of sigma2:  1489.9808858438248
std(beta):  [6.48991779e+00 2.23397691e-03]
std(sigma2):  234.1279084930017
```

# Frekventistický versus bayesovský přístup - co je tedy jinak?

1. Bayesovská regrese nám umožňuje kvantifikovat neurčitost spojenou se znalostí šumu, odhadů atd. Můžeme se vždy podívat na aposteriorní distribuci a říci, jak moc jsme si výsledkem jisti. Čím koncentrovanější aposteriorno, tím méně nejistoty.
2. Oproti frekventismu jsme v bayesovství nuceni si vybrat vhodnou apriorní distribuci. A ta konjugovaná je sice příjemná na počítání, ale nemusí být nutně ideální. Problematika "ideální" apriorní distribuce je v bayesovství často diskutována a byla dlouhou dobu důvodem nevole statistiků vůči bayesovskému aparátu.
3. Apriorní distribuce zajišťuje regularizaci, něco, co klasická statistika moc neumí a vypomáhá si různými heuristikami.
4. Výpočty s bayesovským přístupem jsou často komplikované a jejich analytičnost je spíše výjimkou. To je další z důvodů, proč bayesovství na svůj vrchol čekalo několik století. Ten nastal až s příchodem výkonných počítačů.
5. Oproti frekventismu mají bayesovské výsledky poměrně přímočarou interpretaci. Například rozdíl mezi bayesovskými a frekventistickými intervaly spolehlivosti je markantní.
6. Asymptoticky jsou odhady ekvivalentní.
7. ***Je na uživateli, jaký aparát si vybere. Podstatné je, zda správně interpretuje své výsledky!***

# Doporučená literatura

- J. Anděl: Statistické metody
- S. Chaterjee a A.S. Hadi: Regression analysis by example
- S. Banerjee: Bayesian linear model: Gory details (pdf online)